<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a393aa3b31186284f7827069618babb05f769937933e072096ff497425c675ab
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-01 16:04:15
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: 1.57 L (1.13%)
Current PnL: -21.26 L (-14.11%)
CY Booked + Current PnL: -9.82 L (-6.52%)
-------------------
Total profit:  1.31 L
Total loss:  -22.58 L
-------------------
Total Booked + Current PnL: 17.16 L (13.87%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.15%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.57 L (63.12%)
Deployed:  1.24 C
Current:  1.40 C
CAGR/XIRR %: 7.72%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.55,113685.0,13181.0,15927.0,0.29,13.12,14.01,28.96,79.0,0.83,0.81,51.54,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.60,37.0,M-SC,2.45,83454.0,-17323.0,17417.0,0.33,-17.19,20.87,0.09,245.0,-0.99,0.60,9.64,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,8.71,45.0,H-MC,3.40,187780.0,23110.0,20224.0,-0.94,14.03,10.77,26.31,89.0,1.14,1.34,23.14,X40N,NTT,AC
42,ITC,452.00,-38.77,47.0,H-LC,2.03,198338.0,-1800.0,22690.0,1.01,-0.90,11.44,10.44,4.0,-0.08,1.42,4.61,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.79,42.0,H-LC,3.70,213517.0,3151.0,25622.0,0.34,1.50,12.00,13.68,37.0,0.12,1.53,18.27,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.0,73.25,79.0,H-SC,1.87,158702.0,-20369.0,86096.0,17.08,-11.37,54.25,36.71,135.0,-0.24,1.14,57.46,XR,NTT,BANKS
26,GREENPANEL,537.0,224.94,55.0,M-SC,1.29,145082.0,-37996.0,116980.0,7.72,-20.75,80.63,43.14,230.0,-0.32,1.04,36.47,XY24,NTT,MISC
64,SHALBY,327.0,1246.09,69.0,M-SC,14.29,183056.0,1913.0,43549.0,6.89,1.06,23.79,25.10,235.0,0.04,1.31,46.70,XY24,NTT,HEALTHCARE
73,TATAMOTORS,1065.0,-17.55,60.0,M-LC,2.29,260761.0,-15762.0,125843.0,5.61,-5.70,48.26,39.81,54.0,-0.13,1.87,24.97,XY24,NTT,AUTO
61,SAMMAANCAP,326.0,-188.64,81.0,M-SC,8.40,152622.0,3402.0,140779.0,5.42,2.28,92.24,96.62,208.0,0.02,1.09,65.17,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-47.99,23.0,M-SC,1.19,87930.0,-3568.0,82320.0,-4.88,-3.90,93.62,86.07,223.0,-0.04,0.63,27.69,XR,NTT,DURABLES
3,ALKYLAMINE,4546.37,-12.24,28.0,H-SC,10.68,81678.0,-19285.0,113818.0,-3.59,-19.10,139.35,93.63,148.0,-0.17,0.58,21.66,SR,ATH,CHEMICALS
55,RAJOOENG,143.10,-45.28,20.0,H-SC,14.24,82810.0,-19690.0,60294.0,-3.17,-19.21,72.81,39.61,114.0,-0.33,0.59,0.00,AR,ATH,MISC
81,VAIBHAVGBL,521.00,47.66,44.0,H-SC,5.80,128056.0,-54719.0,165781.0,-2.34,-29.94,129.46,60.77,125.0,-0.33,0.92,14.69,XR,NTT,JEWELLERY
38,INDIGOPNTS,1408.00,102.57,31.0,M-SC,1.51,133002.0,-41557.0,41590.0,-1.65,-23.81,31.27,0.02,221.0,-1.00,0.95,15.69,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.68,44.0,H-MC,2.20,134832.0,-956.0,43713.0,1.90,-0.70,32.42,31.48,107.0,-0.02,0.96,11.33,X40,ATH,INSURANCE
86,WIPRO,420.0,-15.15,41.0,M-LC,5.77,149463.0,-1482.0,110931.0,0.71,-0.98,74.22,72.51,53.0,-0.01,1.07,4.98,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,65.0,H-SC,11.47,130221.0,-12258.0,31774.0,4.06,-8.60,24.40,13.70,163.0,-0.39,0.93,53.70,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,56.0,H-SC,1.58,224298.0,-45369.0,81712.0,1.07,-16.82,36.43,13.47,138.0,-0.56,1.60,13.45,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.60,37.0,M-SC,2.45,83454.0,-17323.0,17417.0,0.33,-17.19,20.87,0.09,245.0,-0.99,0.60,9.64,OX40N,NTT,DURABLES
66,SIS,528.0,2105.70,45.0,H-SC,4.04,89020.0,-22012.0,45623.0,1.13,-19.82,51.25,21.26,156.0,-0.48,0.64,20.28,OX40N,NTT,MISC
18,CERA,9475.0,-22.91,35.0,H-SC,1.61,140438.0,-35465.0,77494.0,-0.85,-20.16,55.18,23.89,149.0,-0.46,1.00,21.29,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1246.09,69.0,M-SC,14.29,183056.0,1913.0,43549.0,6.89,1.06,23.79,25.10,235.0,0.04,1.31,46.70,XY24,NTT,HEALTHCARE
60,SAIL,228.00,47.55,60.0,M-MC,11.51,232332.0,7370.0,160286.0,0.33,3.28,68.99,74.53,192.0,0.05,1.66,36.59,XY24,BTT,STEEL
32,HINDZINC,730.22,28.52,64.0,M-LC,3.86,209387.0,4311.0,108253.0,-0.23,2.10,51.70,54.89,52.0,0.04,1.50,26.70,X5K,ATH,METALS
14,BLUESTARCO,2080.00,8.71,45.0,H-MC,3.40,187780.0,23110.0,20224.0,-0.94,14.03,10.77,26.31,89.0,1.14,1.34,23.14,X40N,NTT,AC
84,VOLTAS,1530.00,-3.18,43.0,H-MC,2.84,203025.0,11283.0,26474.0,0.99,5.88,13.04,19.69,99.0,0.43,1.45,13.08,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.55,113685.0,13181.0,15927.0,0.29,13.12,14.01,28.96,79.0,0.83,0.81,51.54,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,9.45,191681.0,11873.0,100537.0,0.99,6.60,52.45,62.52,88.0,0.12,1.37,42.94,XR,NTT,BANKS
32,HINDZINC,730.22,28.52,64.0,M-LC,3.86,209387.0,4311.0,108253.0,-0.23,2.10,51.70,54.89,52.0,0.04,1.50,26.70,X5K,ATH,METALS
37,INDIAMART,4850.92,-54.45,33.0,H-SC,10.02,127142.0,3806.0,129952.0,1.36,3.09,102.21,108.45,119.0,0.03,0.91,26.13,AR,ATH,MISC
86,WIPRO,420.00,-15.15,41.0,M-LC,5.77,149463.0,-1482.0,110931.0,0.71,-0.98,74.22,72.51,53.0,-0.01,1.07,4.98,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.55,28.0,H-LC,5.83,205550.0,-46218.0,99425.0,-0.60,-18.36,48.37,21.13,27.0,-0.46,1.47,10.56,X40,ATH,PAINTS
83,VBL,671.64,-21.55,32.0,H-LC,9.26,282602.0,-33240.0,145907.0,-0.17,-10.52,51.63,35.67,5.0,-0.23,2.02,1.87,X40N,ATH,FMCG
74,TCS,4476.75,-31.26,32.0,H-LC,12.39,273935.0,-72021.0,146884.0,0.89,-20.82,53.62,21.64,1.0,-0.49,1.96,0.89,X40,ATH,IT
30,HCLTECH,1943.91,-4.52,34.0,H-LC,10.24,218152.0,-23772.0,87043.0,0.32,-9.83,39.90,26.15,8.0,-0.27,1.56,3.09,X40,ATH,IT
44,JIOFIN,387.00,-5.04,38.0,H-LC,12.90,258043.0,-9322.0,74781.0,2.34,-3.49,28.98,24.48,48.0,-0.12,1.85,49.57,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-6.94,43.0,H-LC,1.62,230535.0,-15616.0,94819.0,-0.68,-6.34,41.13,32.18,16.0,-0.16,1.65,27.41,X200,ATH,IT
65,SIEMENS,4671.5,-2.83,44.0,H-LC,1.98,156075.0,-30020.0,77507.0,-0.25,-16.13,49.66,25.51,15.0,-0.39,1.12,15.02,AR,ATH,ELECTRICAL
42,ITC,452.0,-38.77,47.0,H-LC,2.03,198338.0,-1800.0,22690.0,1.01,-0.90,11.44,10.44,4.0,-0.08,1.42,4.61,X40,NTT,FMCG
57,RELIANCE,1533.0,-14.79,42.0,H-LC,3.70,213517.0,3151.0,25622.0,0.34,1.50,12.00,13.68,37.0,0.12,1.53,18.27,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-22.18,42.0,H-LC,3.96,207281.0,2781.0,38637.0,1.68,1.36,18.64,20.25,10.0,0.07,1.48,10.21,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,32.0,H-LC,12.39,273935.0,-72021.0,146884.0,0.89,-20.82,53.62,21.64,1.0,-0.49,1.96,0.89,X40,ATH,IT
83,VBL,671.64,-21.55,32.0,H-LC,9.26,282602.0,-33240.0,145907.0,-0.17,-10.52,51.63,35.67,5.0,-0.23,2.02,1.87,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.52,34.0,H-LC,10.24,218152.0,-23772.0,87043.0,0.32,-9.83,39.90,26.15,8.0,-0.27,1.56,3.09,X40,ATH,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.03,198338.0,-1800.0,22690.0,1.01,-0.90,11.44,10.44,4.0,-0.08,1.42,4.61,X40,NTT,FMCG
40,INFY,2275.00,-21.03,39.0,H-LC,8.19,307955.0,-4933.0,176612.0,0.28,-1.58,57.35,54.87,3.0,-0.03,2.20,4.97,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.83,44.0,H-LC,1.98,156075.0,-30020.0,77507.0,-0.25,-16.13,49.66,25.51,15.0,-0.39,1.12,15.02,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,47.0,H-LC,2.03,198338.0,-1800.0,22690.0,1.01,-0.90,11.44,10.44,4.0,-0.08,1.42,4.61,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.55,28.0,H-LC,5.83,205550.0,-46218.0,99425.0,-0.60,-18.36,48.37,21.13,27.0,-0.46,1.47,10.56,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.18,42.0,H-LC,3.96,207281.0,2781.0,38637.0,1.68,1.36,18.64,20.25,10.0,0.07,1.48,10.21,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.79,42.0,H-LC,3.70,213517.0,3151.0,25622.0,0.34,1.50,12.00,13.68,37.0,0.12,1.53,18.27,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,32.0,H-LC,12.39,273935.0,-72021.0,146884.0,0.89,-20.82,53.62,21.64,1.0,-0.49,1.96,0.89,X40,ATH,IT
40,INFY,2275.00,-21.03,39.0,H-LC,8.19,307955.0,-4933.0,176612.0,0.28,-1.58,57.35,54.87,3.0,-0.03,2.20,4.97,X40,BTT,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.03,198338.0,-1800.0,22690.0,1.01,-0.90,11.44,10.44,4.0,-0.08,1.42,4.61,X40,NTT,FMCG
83,VBL,671.64,-21.55,32.0,H-LC,9.26,282602.0,-33240.0,145907.0,-0.17,-10.52,51.63,35.67,5.0,-0.23,2.02,1.87,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,46.0,H-LC,7.69,249552.0,-12067.0,131289.0,0.49,-4.61,52.61,45.57,7.0,-0.09,1.79,6.19,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,40.33,79526.0,-34023.0,74071.0,2.45,-29.96,93.14,35.27,268.0,-0.46,0.57,94.64,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,43.0,H-SC,12.41,92055.0,-5925.0,27524.0,1.39,-6.05,29.90,22.05,152.0,-0.22,0.66,33.81,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.55,113685.0,13181.0,15927.0,0.29,13.12,14.01,28.96,79.0,0.83,0.81,51.54,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,65.0,H-SC,11.47,130221.0,-12258.0,31774.0,4.06,-8.60,24.40,13.70,163.0,-0.39,0.93,53.70,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-188.64,81.0,M-SC,8.40,152622.0,3402.0,140779.0,5.42,2.28,92.24,96.62,208.0,0.02,1.09,65.17,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-188.64,81.0,M-SC,8.40,152622.0,3402.0,140779.0,5.42,2.28,92.24,96.62,208.0,0.02,1.09,65.17,XY25,NTT,FINANCE
64,SHALBY,327.00,1246.09,69.0,M-SC,14.29,183056.0,1913.0,43549.0,6.89,1.06,23.79,25.10,235.0,0.04,1.31,46.70,XY24,NTT,HEALTHCARE
69,SURYODAY,240.00,73.25,79.0,H-SC,1.87,158702.0,-20369.0,86096.0,17.08,-11.37,54.25,36.71,135.0,-0.24,1.14,57.46,XR,NTT,BANKS
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.55,113685.0,13181.0,15927.0,0.29,13.12,14.01,28.96,79.0,0.83,0.81,51.54,MH,ATH,METALS
60,SAIL,228.00,47.55,60.0,M-MC,11.51,232332.0,7370.0,160286.0,0.33,3.28,68.99,74.53,192.0,0.05,1.66,36.59,XY24,BTT,STEEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.03
1,25,43.12
2,50,74.87


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.25
LC    32.07
MC    23.69
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.96
X40      14.46
X40N     11.23
XR       11.10
XY25     10.45
AR        8.11
OX40N     7.78
X200      1.65
MH        1.61
X5K       1.50
SR        1.16
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.01
H-LC    25.47
H-MC    20.42
M-SC    13.80
M-LC     5.57
M-MC     2.97
L-SC     1.44
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.82,-6.18,41.23
IT,12.48,-20.50,86.21
FINANCE,10.43,-15.94,68.60
MISC,6.96,-18.27,84.02
BANKS,6.44,-10.90,69.24
PAINTS,5.55,-18.78,36.49
ELECTRICAL,5.34,-10.49,49.25
INSURANCE,3.75,-5.42,44.82
AUTO,3.58,-15.00,69.98


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3249321.0,22
XR,1359880.0,14
AR,1198892.0,9
X40,827107.0,10
X40N,636209.0,9
OX40N,560600.0,10
XY25,475481.0,7
SR,259143.0,2
X5K,108253.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3738879.0,29
M-SC,1505332.0,17
H-LC,1260229.0,15
H-MC,1250469.0,15
M-LC,372603.0,4
M-MC,368093.0,2
L-SC,261171.0,3
L-MC,60240.0,1
L-LC,40507.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1232154.0      6
M-SC       XY24       848129.0      7
H-SC       AR         820853.0      5
           XR         784695.0      7
H-MC       XY24       572745.0      4
H-LC       X40        532654.0      5
M-MC       XY24       368093.0      2
H-SC       X40N       313827.0      4
           SR         259143.0      2
           OX40N      256316.0      4
H-LC       X40N       235688.0      3
H-MC       X40        219094.0      4
H-LC       AR         208796.0      2
H-MC       XY25       180704.0      2
L-SC       XR         172523.0      2
M-SC       AR         169243.0      2
M-LC       XY24       153419.0      2
M-SC       OX40N      140868.0      4
           XY25       140779.0      1
           XR         130954.0      2
H-LC       XY25       113491.0      3
M-LC       XR         110931.0      1
           X5K        108253.0      1
H-MC       XR         100537.0      1
H-LC       X200        94819.0      1
L-SC       OX40N       88648.0      1
H-MC       X40N        86694.0      2
M-SC       X40         75359.0      1
H-LC       XY24        74781.0      1
H-MC       OX40N       74768.0      1
H-SC       MH          71891.0      1
L-MC       XR          60240.0      1
L-LC       XY25        40507.0      1
H-MC       MH          15927.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
